# Preamble

The following notebook demonstrates the tuning procedure for the ANN models. This demonstration uses 30 lags and 1-day ahead predictions.  

# Import packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

Import tensorflow and keras packages

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
import tensorflow as tf

# Import data

Read the data using Pandas

In [ ]:
df = pd.read_csv("feature_sel_1_day.csv", index_col=0, parse_dates=[0])

Create the split using a *70% / 30%* split

In [ ]:
train_size = 0.7
n = len(df)
train_ind = int(n*train_size)
train_df = df[:train_ind]
test_df = df[train_ind:]

# Tuning

Perform hyperparameter tuning of an ANN model using the `Keras tuner` package and Bayesian optimization.

Import packages

In [ ]:
from keras_tuner import HyperParameters as hp

In [ ]:
from keras import backend as backend

## Building the model

In order to perform a hyperparameter search using the Keras Tuner package, we have to create a function that details how the model shall be built. This entails setting the search space and creating the architecture. In our thesis we ran a separate search for each different variation of the Bayesian Neural Networks and LSTM models, but these can be condensed into one single tuning session by letting the type of prior and posterior be part of the search space. We will not demonstrate how this can be done, as this makes the search space more complex, thus confuscating the inference of model performances. 

### LSTM model
Create function for building an LSTM model for the tuning procedure.

Note that the tuning procedure can be memory intensive if a GPU is utilized. To alleviate this issue, the `backend.clear_session()` command may be used. Further alterations are applied in each trial of the tuner. 

In [ ]:
def build_LSTM_model(hp):
    backend.clear_session()  
    model = Sequential()
    # Input layer
    first_layer = hp.Int("input_unit_neurons", min_value=16, max_value=64, step=16)
    model.add(LSTM(first_layer, return_sequences=True, input_shape=(input_width, num_features)))
    
    # Hidden
    use_hidden_layer = hp.Boolean("use_hidden_layer")
    with hp.conditional_scope("use_hidden_layer", [True]):
        if use_hidden_layer:
            hidden_layer = hp.Int("hidden_layer_neurons", min_value=32, max_value=128, step=32)
            model.add(LSTM(hidden_layer, return_sequences=True))
    
    # Final LSTM layer    
    final_layer = hp.Int("last_unit_neurons", min_value=16, max_value=128, step=16)
    model.add(LSTM(final_layer))
    
    # Add dropout layer
    model.add(Dropout(hp.Float('Dropout_rate', min_value=0, max_value=0.5, step=0.1)))
    
    # Final output layer
    model.add(Dense(units=1))
    
    # 
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    metrics = ["mse", "mae", "mape", tf.keras.metrics.RootMeanSquaredError()]
    model.compile(loss='mse', optimizer=Adam(learning_rate=learning_rate), metrics = metrics)
    return model

### MC Dropout model
The equivalent model builder for a Monte Carlo Dropout model. This is very similar to the LSTM model, albeit with more dropout layers.

In [ ]:
def build_mc_dropout_model(hp):
    backend.clear_session()
    model = Sequential()
    # LSTM aspect
    # Input layer
    model.add(LSTM(64, return_sequences=True, input_shape=(input_width, num_features)))

    # First dropout layer
    first_dropout_rate = hp.Float("first_dropout_rate", min_value=0.1, max_value=0.6, step=0.1)
    model.add(Dropout(rate=first_dropout_rate))
  
    # Final LSTM layer    
    model.add(LSTM(96))

    # Second dropout layer
    second_dropout_rate = hp.Float("second_dropout_rate", min_value=0.1, max_value=0.6, step=0.1)
    model.add(Dropout(rate=second_dropout_rate))
    
    # Dense output
    model.add(Dense(units=1))

    learning_rate =  hp.Float('learning_rate', min_value=0.0001, max_value=0.05)
    optimizer = Adam(learning_rate=learning_rate)      
    metrics = ["mse", "mae", "mape", tf.keras.metrics.RootMeanSquaredError(), tf.keras.losses.Huber()]
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics = metrics)
    return model

### Diagonal Normal Bayesian Neural Network
The model builder function for a diagonal BNN model. 

Note the prior and posterior functions, which determines the type of probability function to be used for these two parameters. In this case, a non-trainable diagonal normal prior is employed where only the `loc` (or mean) and `scale` (or standard deviation) is specified. For the sake of simplicity a mean of zero is used, leaving only the variance as the adjustable parameter. 

For the posterior distribution, the rescaling factor is tuned. This parameter scales down the variance to achieve more stable training performance.

In [ ]:
def build_bayesian_diag_normal_model(hp):
    backend.clear_session()
    model = Sequential()
    # LSTM aspect
    # Input layer
    model.add(LSTM(64, return_sequences=True, input_shape=(input_width, num_features)))

    # Final LSTM layer    
    model.add(LSTM(96))

    # Bayesian aspect
    
    # Scale of prior
    # USING DIAGONAL NORMAL!
    normal_scale = hp.Float("normal_scale", min_value=0.1, max_value=2)
    def prior_tune_normal(kernel_size, bias_size, dtype = None):
        n = kernel_size + bias_size # num of params
        sc = tf.ones(n)*normal_scale
        return Sequential([
            tfp.layers.DistributionLambda(
                lambda t: tfd.MultivariateNormalDiag(loc = tf.zeros(n), scale_diag = sc)
                )                     
          ])
    prior_func = prior_tune_normal

    # Scale of posterior
    posterior_scale = hp.Float('posterior_scale', min_value=0.0001, max_value=0.1)
    def posterior_tune(kernel_size, bias_size, dtype=None):
        n = kernel_size + bias_size
        c = np.log(np.expm1(1.))
        return tf.keras.Sequential([
          tfp.layers.VariableLayer(2 * n, dtype=dtype),
          tfp.layers.DistributionLambda(lambda t: tfd.Independent(
              tfd.Normal(loc=t[..., :n],
                         scale = 1e-3 + posterior_scale * tf.nn.softplus(c + t[..., n:])),
              reinterpreted_batch_ndims=1)),
        ])
    
    # Give opportunity of adding one more dense variational layer
    # Hidden DenseVariational layer
    use_hidden_vi_layer = hp.Boolean("use_hidden_vi_layer")
    with hp.conditional_scope("use_hidden_vi_layer", [True]):
        if use_hidden_vi_layer:
            hidden_layer_units = hp.Int("hidden_vi_layer_neurons", min_value=8, max_value=64)
            model.add(tfp.layers.DenseVariational(
                hidden_layer_units, make_posterior_fn=posterior_tune, make_prior_fn=prior_func,
                kl_weight=1/num_samples
            ))
    
    
    model.add(
        tfp.layers.DenseVariational(
            1, make_posterior_fn=posterior_tune, make_prior_fn=prior_func, 
            kl_weight=1/num_samples
        )    
    )
    
    learning_rate =  hp.Float('learning_rate', min_value=0.0001, max_value=0.1)
    optimizer = Adam(learning_rate=learning_rate)      
    metrics = ["mse", "mae", "mape", tf.keras.metrics.RootMeanSquaredError(), tf.keras.losses.Huber()]
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics = metrics)
    return model

While the above demonstrates a Diagonal Normal hybrid model builder, small alterations can be made to create the model builder function for the other models:
* If a `Laplace prior` is used, simply replace the `Normal` part of the Distribution Lambda with a `tfd.Laplace` function.
* If a `Pure Bayesian` model is used, replace the first LSTM layers with the desired number of Dense Variational Layers. Apply the same logic as the optional hidden layer used above to search for the number of hidden layeres to utilize.

### Gaussian Scale Mixture Model
The Scale Mixture model is somewhat more complicated, as the current (as of June 2023) Tensorflow Probability framework does not allow for a simple implementation of this. The next cell demonstrates how this prior can be used by extending on the base Dense Variational Class. First, a custom Dense variational Class is built.

In [ ]:
# Define custom DVI class able to handle scale gaussian mixture priors
class Custom_dvi(tfp.layers.DenseVariational):
    """
    An extension of the Tensorflow Probability Dense Variational Class to allow for having Gaussian Scale Mixture priors. 
    """

    def __init__(self, comp_weight, first_scale, second_scale, **kwargs) -> None:
        super().__init__(**kwargs)
        self.comp_weight = comp_weight
        self.first_scale = first_scale
        self.second_scale = second_scale
        self._kl_divergence_fn = _make_kl_divergence_penalty(
        use_exact_kl=kwargs.get('kl_use_exact', False), weight=kwargs.get('kl_weight', 1335), 
        comp_weight=self.comp_weight, first_prior_scale=self.first_scale, second_prior_scale=self.second_scale
        )

    def call(self, inputs):
        dtype = tf.as_dtype(self.dtype or tf.keras.backend.floatx())
        inputs = tf.cast(inputs, dtype, name='inputs')

        q = self._posterior(inputs)
        r = self._prior(inputs)
        self.add_loss(
            self._kl_divergence_fn(q, r))

        w = tf.convert_to_tensor(value=q)
        prev_units = self.input_spec.axes[-1]
        if self.use_bias:
            split_sizes = [prev_units * self.units, self.units]
            kernel, bias = tf.split(w, split_sizes, axis=-1)
        else:
            kernel, bias = w, None

        kernel = tf.reshape(kernel, shape=tf.concat([
            tf.shape(kernel)[:-1],
            [prev_units, self.units],
        ], axis=0))
        outputs = tf.matmul(inputs, kernel)

        if self.use_bias:
            outputs = tf.nn.bias_add(outputs, bias)

        if self.activation is not None:
            outputs = self.activation(outputs)  # pylint: disable=not-callable

        return outputs


def _make_kl_divergence_penalty(
    use_exact_kl=False,
    test_points_reduce_axis=(),  # `None` == "all"; () == "none".
    test_points_fn=tf.convert_to_tensor,
    weight=None,
    comp_weight=None,
    first_prior_scale=None,
    second_prior_scale=None
):
    """Creates a callable computing `KL[a,b]` from `a`, a `tfd.Distribution`."""

    if use_exact_kl:
        kl_divergence_fn = kullback_leibler.kl_divergence
    else:
        def kl_divergence_fn(distribution_a, distribution_b):
            z = test_points_fn(distribution_a)

            return tf.reduce_mean(
              distribution_a.log_prob(z) - log_mixture_prior_prob(z, comp_weight, first_prior_scale, second_prior_scale),
              axis=test_points_reduce_axis)

  # Closure over: kl_divergence_fn, weight.
  def _fn(distribution_a, distribution_b):
    """Closure that computes KLDiv as a function of `a` as in `KL[a, b]`."""
    with tf.name_scope('kldivergence_loss'):
        kl = kl_divergence_fn(distribution_a, distribution_b)
        if weight is not None:
            kl = tf.cast(weight, dtype=kl.dtype) * kl
      # Losses appended with the model.add_loss and are expected to be a single
      # scalar, unlike model.loss, which is expected to be the loss per sample.
      # Therefore, we reduce over all dimensions, regardless of the shape.
      # We take the sum because (apparently) Keras will add this to the *post*
      # `reduce_sum` (total) loss.
      # TODO(b/126259176): Add end-to-end Keras/TFP test to ensure the API's
      # align, particularly wrt how losses are aggregated (across batch
      # members).
        return tf.reduce_sum(kl, name='batch_total_kl_divergence')

    return _fn

def log_mixture_prior_prob(w, comp_weight, first_prior_scale, second_prior_scale):
    # Applies the formula of Blundell et al 2015 
    comp_1_dist = tfp.distributions.Normal(0.0, first_prior_scale)
    comp_2_dist = tfp.distributions.Normal(0.0, second_prior_scale)
    comp_1_weight = comp_weight   
    return tf.reduce_sum(tf.math.log(comp_1_weight * comp_1_dist.prob(w) + (1 - comp_1_weight) * comp_2_dist.prob(w)))




The following function builds the Scale Gaussian Mixture model

In [ ]:
from custom_dense_variational import Custom_dvi
def build_bayesian_scale_mixture_model(hp):
    backend.clear_session()
    model = Sequential()
    # LSTM aspect
    # Input layer
    model.add(LSTM(64, return_sequences=True, input_shape=(input_width, num_features)))

    # Final LSTM layer    
    model.add(LSTM(96))

    # Bayesian aspect

    # Using Scale Gaussian mixture prior!
    # Each density is zero mean, but variance of first greater than second
    # Need to choose: weighting and scales
    # Weigting of components
    components_weight = hp.Choice("mixture_weight", [0.25, 0.5, 0.75])
    # Variances of components
    first_prior_scale = hp.Choice("first_prior_scale", [0.1, 0.25, 0.5, 0.75, 1.])
    second_prior_scale = hp.Choice("second_prior_scale", [0.0001, 0.00025, 0.0005, 0.00075, 0.001])
    

    def prior_mvmg(kernel_size, bias_size, dtype=None):
        n = kernel_size + bias_size
        return tf.keras.Sequential([
          tfp.layers.DistributionLambda(lambda t: tfd.MixtureSameFamily(
              mixture_distribution=tfd.Categorical(
              probs=[components_weight, (1-components_weight)]),
              components_distribution=tfd.Normal(
                  loc=[0, 0],       # Zero mean as described in paper
                  scale=[first_prior_scale, second_prior_scale] # 
              ) 
          ))          
        ])
    prior_func = prior_mvmg

    # Scale of posterior
    posterior_scale = hp.Float('posterior_scale', min_value=0.0001, max_value=0.1)
    def posterior_tune(kernel_size, bias_size, dtype=None):
        n = kernel_size + bias_size
        c = np.log(np.expm1(1.))
        return tf.keras.Sequential([
          tfp.layers.VariableLayer(2 * n, dtype=dtype),
          tfp.layers.DistributionLambda(lambda t: tfd.Independent(
              tfd.Normal(loc=t[..., :n],
                         scale = 1e-3 + posterior_scale * tf.nn.softplus(c + t[..., n:])),
              reinterpreted_batch_ndims=1)),
        ])
    
    # Give opportunity of adding one more dense variational layer
    # Hidden DenseVariational layer
    use_hidden_vi_layer = hp.Boolean("use_hidden_vi_layer")
    with hp.conditional_scope("use_hidden_vi_layer", [True]):
        if use_hidden_vi_layer:
            hidden_layer_units = hp.Int("hidden_vi_layer_neurons", min_value=2, max_value=64)
            model.add(tfp.layers.DenseVariational(
                hidden_layer_units, make_posterior_fn=posterior_tune, make_prior_fn=prior_func,
                kl_weight=1/num_samples
            ))
    
    model.add(
        Custom_dvi(
            units=1, make_posterior_fn=posterior_tune, make_prior_fn=prior_func, 
            kl_weight=1/num_samples,
            comp_weight=components_weight, first_scale=first_prior_scale,
            second_scale=second_prior_scale
        )    
    )
    
    learning_rate =  hp.Float('learning_rate', min_value=0.0001, max_value=0.01)
    optimizer = Adam(learning_rate=learning_rate)      
    metrics = ["mse", "mae", "mape", tf.keras.metrics.RootMeanSquaredError(), tf.keras.losses.Huber()]
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics = metrics)
    return model

## Trial function

As we use a cross-validation approach, we implemented a custom trial function that specifies the behavior of the search for each trial. The BNNs requires drawing Monte Carlo (MC) samples of the models to estimate their performance, exemplified in the for loop. For the LSTM tuning, this line can be dropped. The MC Dropout model requires setting the training parameter to True to achieve Dropout behavior when making predictions. 

The code runs through each specified window, builds the model, trains it and makes predictions. Then, the predictions are evaluated according to the desired evaluation metrics.

In [ ]:
from time import sleep

# Specify evaluation metrics as a dictionary
rmse_ = tf.keras.metrics.RootMeanSquaredError()
huber_ = tf.keras.losses.Huber()
ms = {
    "MSE": tf.keras.metrics.mse, "MAE": tf.keras.metrics.mae, "MAPE": tf.keras.metrics.mape, 
    "RMSE": rmse_, "Huber": huber_
}
# Specify the number of epochs to run, and is using a BNN, the number of MC samplings of models to use
num_epochs = 50
num_mc_samples = 100
from sklearn.metrics import mean_squared_error
class BayesTuner_Dropout(kt.engine.tuner.Tuner):
    def run_trial(self, trial, folder, callbacks, *args, **kwargs):
        # Create dictionary to store MSE results
        fold_results = {"val_mse":[]}
        print("Going over folds...")
        mse_list = []
        for i, fold in enumerate(folder):  # Iterate over windows
            print(f"Starting fold {i+1}")
            # Get the Tensorflow Map Dataset instance of the current window
            train, val = fold
            # Build model using the current trials' set of hyperparameters
            model = self.hypermodel.build(trial.hyperparameters)
            # Fit the model
            hist = model.fit(train, epochs=num_epochs, validation_data=val, verbose=0, callbacks=callbacks)
            # Print the number of epochs
            antall_epochs = len(hist.history["loss"])
            print(f"Finished training in {antall_epochs} epochs")
            epochs_key = f"num_epochs_{i}"
            fold_results[epochs_key] = antall_epochs
            # Make first prediction to get size of array
            print("Making predictions")
            # Convert the validation data into array as this is required if using the __call__ method for predictions 
            # in a Keras model
            x_val = np.concatenate([x for x, y in val], axis=0)
            # Make first prediction
            pred = model(x_val, training=True).numpy().flatten()  # Flatten to prevent (num, 1)
            # If tuning an LSTM model, the folowing code is not necessary. Simply use the prediction, and evaluate it for each 
            # desired metric
            # Save predictions in a prediction array for each Monte Carlo sampling of the model
            predictions = np.zeros(shape=(num_mc_samples, pred.shape[0]))
            # Run 100 Monte Carlo sampling of the model to estimate the epistemic uncertainty
            for j in range(1, num_mc_samples):  # Run 100 MC evaluations to get Model performance
                # To achieve Dropout behavior during predictions, the training argument is set to True
                predictions[j] = model(x_val, training=True).numpy().flatten()
            # Get mean and standard deviation of predictions
            predictions_mean = predictions.mean(axis=0)
            predictions_std = predictions.std(axis=0)
            
            # Evaluate predictions
            # Get y_value
            val_y = np.concatenate([y for x, y in val], axis=0).reshape(-1)
            # Get losses for all metrics
            for key, metric in ms.items():
                score = metric(val_y, predictions_mean).numpy()
                print(f"{key}: {score:.5f}")
                new_key = f"{key}_{i}"
                fold_results[new_key] = score
                #fold_results[KeyedRef]
                #fold_results[key].append(score)
            mse_list.append(mean_squared_error(val_y, predictions_mean))


            # Compute and save correlation
            corr = np.corrcoef(np.abs(val_y-predictions_mean), predictions_std)[0, 1]
            fold_results[f"Corr_{i}"] = corr
            print(f"Standard deviation/residuals correlation: {corr:.3f}")
            
            # Save spread of standard deviation to assess how volatile the 
            # performance is
            std_spread = predictions_std.max() - predictions_std.min()
            print(f"Standard deviation spread: {std_spread:.5f}")
            fold_results[f"StdDev_{i}"] = std_spread
            # The model can be saved, but this eats up a lot of memory leading us to drop this feature of the
            # tuning procedure
            #self.save_model(trial.trial_id, model)
            print()
            # Delete the model to free up memory
            del model
        # Compute means for objective value 
        fold_results["val_mse"] = np.mean(mse_list)
        
        # Update trial
        self.oracle.update_trial(
            trial.trial_id,
            fold_results
        )

Create `WindowGenerator` object using the `window_generator` module

In [ ]:
from window_generator import WindowGenerator

Create the windows. 30 lags are included in this demonstration

In [ ]:
input_width = 30
window = WindowGenerator(
    input_width=input_width, label_width=1, shift=1, label_columns=['VIX'], 
    train_df=train_df, test_df=test_df, scale=MinMaxScaler)
window

Store the folds using as list of Tensorflow `Map Datasets` 

In [ ]:
folder = window.folds

Run the search for 100 trials. Here, we use a `Callback` that stops the training if the model performance does not improve after a set number of epochs. The `patience` parameter controls this behavior, and the `start_from_epoch` allows the models to warm up before applying the callback. As BNNs can vary greatly for the initial epochs, this parameter helps prevent premature stopping. 

In [ ]:
overwrite=True
tuner = CVTuner_LSTM(oracle=kt.oracles.BayesianOptimizationOracle(objective='val_mse', max_trials=100), hypermodel=build_LSTM_model, directory="baseline", project_name="lstm_regular", overwrite=overwrite)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_mse', patience=10, start_from_epoch=10)

tuner.search(folder=folder, callbacks=[stop_early])

# Display tuner results

Create dataframe for storing the results for each fold

In [ ]:
mds = [f"Model_{i+1}" for i in range(5)]
fds = ["1", "2", "3", "4", "5", "Mean"]
iterables = [mds, fds]
idx = pd.MultiIndex.from_product(iterables, names=["Model", "Fold"])

In [ ]:
tunres = pd.DataFrame(index=idx, columns=["MSE", "MAE", "Huber", "Std"])

Iterate over each of the ten best models, and save their acquired metrics. Additionally, print the model hyperparameters and performances.

In [ ]:
# Create dictionary of overall best values
best_values = {i:[np.inf, -1, 100] for i in tuner.oracle.get_best_trials()[0].metrics.metrics.keys()}

# Get corr values and mse values for each fold
mses = [f"MSE_{i}" for i in range(5)]
corrs = [f"Corr_{i}" for i in range(5)]
maes = [f"MAE_{i}" for i in range(5)]
hubers =  [f"Huber_{i}" for i in range(5)]
stds = [f"StdDev_{i}" for i in range(5)]

# Iterate over 10 best trials
for i, trial in enumerate(tuner.oracle.get_best_trials(num_trials=10)):
    print(f"Model rank: {i+1}\tTrial id: {trial.trial_id}")
    print(f"MSE: {trial.score:.4f}")
    print()
    j = 0
    for ms, corr, ma, hub, st in zip(mses, corrs, maes, hubers, stds):
        mval = trial.metrics.get_last_value(ms)
        cval = trial.metrics.get_last_value(corr)
        maeval = trial.metrics.get_last_value(ma)
        hubval = trial.metrics.get_last_value(hub)
        stval = trial.metrics.get_last_value(st)
        
        fd = (f"Model_{i+1}", f"{j+1}")
        tunres.loc[fd, :] = [mval, maeval, hubval, stval]

        print(f"Fold {j}:\tMSE: {mval:.5f}\tCorr: {cval:.4f}\tMAE: {maeval:.4f}\tHuber: {hubval:.4f}\tStd: {stval:.4f}")
        j += 1
    print()
    print(trial.hyperparameters.values)
    print("-"*40)
print()

Save the mean value for each metric over all folds

In [ ]:
for m in tunres.index.get_level_values("Model").unique():
    tunres.loc(axis=0)[m, "Mean"] = tunres.loc(axis=0)[m, fds[:-1]].mean(axis=0)

## Display best performance according to each metric
For every metric, the values for each of the ten best models are sorted to show their corresponding ranking. For brevity purposes, only the `MSE`, `MAE`, and `Huber` metrics are used here. Additionally, only the mean performance of all folds are demonstrated. The `key` argument in the below functions can be used to inspect the performance for specific folds

In [ ]:
tunres.xs(key="Mean", level="Fold").sort_values(by="MSE")

In [ ]:
tunres.xs(key="Mean", level="Fold").sort_values(by="MAE")

In [ ]:
tunres.xs(key="Mean", level="Fold").sort_values(by="Huber")